Aggregation and Composition: in bot cases, one object has a reference to another and delegates some work to it. Using inheritance, the object itself is able to the that work, inheriting behavior from its superclass.

# Global Object Pattern

- Basically instantiating objects at import time and assigning names in the global scope. 
- Avoid import-time I/O. This can cause errors during the import time. Put these operations when they are actually required from the package by the client.

# Prebound Method Pattern

**Advantage**: users can use the methods as stand-alone functions, but they secretly share a state.

**Be careful** about the type of object you are instantiating (classes with "heavy" constructors, like using databases, creating files, etc). Maybe be more explicit about some of the object construction steps (using an extra setup() method could be a good idea).  

**Tip** evene when you have a large number of methods, assign them explicitly (avoid using a for loop for this).

In [ ]:
from datetime import datetime

class Random8(object):
    def __init__(self):
        self.set_seed(datetime.now().microsecond % 255 + 1)

    def set_seed(self, value):
        self.seed = value

    def random(self):
        self.seed, carry = divmod(self.seed, 2)
        if carry:
            self.seed ^= 0xb8
        return self.seed

# Instantiate your class at the top level of your module
# Private name (with _) to the instance
_instance = Random8()

# Assign a copy to each of the object's methods (global namespace)
random = _instance.random
set_seed = _instance.set_seed

# Sentinel Object Pattern

- Sometimes it is necessary to differentiate between an argument that has not been provided, and an argument with the value None.
- A **sentinel** is an object that has no particular meaning except to signal the end or a special condition.
- Philosophy: the object's identity, not its value, is to be used by the surrounding code.

Example using object (base class in Python 3) as a sentinel: 

In [3]:
sentinel = object()

In [4]:
class Field:
    def __init__(self, default=sentinel):
        self.value = default
        
    def get(self):
        if self.value is sentinel:
            raise ValueError("this field has no value!")

In [6]:
eula_accepted = Field()
eula_accepted.get()

ValueError: this field has no value!

In [7]:
eula_accepted = Field(default=None)
eula_accepted.get()

The sentinel defined above does not a very good representation:

In [8]:
repr(sentinel)

'<object object at 0x000002663A271190>'

An alternative is defining a simple class for that:

In [9]:
class NotSet:
    def __repr__(self):
        return 'NotSet'

In [10]:
sentinel = NotSet()
repr(sentinel)

'NotSet'

# Adapter Pattern

- **Adapter**: object that converts the interface of one object so that another object can understand it. 
- This pattern lets you create a middle-layer class thar serves as a translator between your code and a legacy class, 3rd-party class or any other class with a weird interface.
- **Single Responsability Principle**: you can separate interface/data conversion from the primary business logic.
- **Open/Closed Principle**: you can introduce new types of adaptors without breaking the existing client code.
- **Attention**: depending on the complexity added, sometimes it is simpler to modify the service class.

In [111]:
class Target:
    """
    This is the original interface. It returns the column sum of
    an exponentiated DataFrame. 
    """
    def __init__(self, power=1):
        self.power = power
    
    def pd_sum_power(self, df) -> pd.DataFrame:
        return (df**self.power).sum()
    
class Service:
    """
    This defines the new interface that the client wants to use.
    In this case, it gives the column sum for a numpy array.
    The problem here is that we want a DataFrame instead of a 
    numpy array as result.
    """
    def __init__(self, power=1):
        self.power = power
    
    def np_sum_power(self, np_array) -> pd.DataFrame:
        return np.sum((np_array**self.power), axis=0)
        
class Adapter:
    """
    This class serves as an adaptor between target and service.
    Most of the work must be performed by the service instance.
    It should focus on interface/data conversion.
    """
    def __init__(self, service):
        self.service = service
        
    def pd_sum_power(self, np_array):
        np_sum = self.service.np_sum_power(np_array)
        return pd.DataFrame(np_sum, index=("colA", "colB")).iloc[:, 0]
        
def client_code(target, data):
    return target.pd_sum_power(data)

In [112]:
import pandas as pd
import numpy as np

df = pd.DataFrame(dict(colA=[1,2,3,4], colB=[1,2,3,4]))
np_array = np.array([[1,2,3,4], [1,2,3,4]]).T

Using DataFrames (like the original interface was intended to be used):

In [117]:
target = Target(power=4)

In [118]:
client_code(target, df)

colA    354
colB    354
dtype: int64

Using a Numpy array and the adapter, we get the same result:

In [119]:
adapter = Adapter(Service(power=4))

In [120]:
client_code(adapter, np_array)

colA    354
colB    354
Name: 0, dtype: int32

# Bridge Pattern

- Two layers of abstractions between classes and one classes that will be dependent upon the other (example of tv and remote controls).
- Progressively adding functionality while separating out major differences using abstract classes.
- The **Bridge** patterns connects the abstractions. After this, each separate hierarchy can be developed separetely.

In [133]:
class Control:
    
    def __init__(self, device):
        self.device = device
    
    def on_off(self):
        self.device.on_off()
    
class ControlVolume(Control):
    
    def adjust_volume(self, delta):
        self.device.adjust_volume(delta)
    
    
class Device:
    
    def __init__(self):
        self.on_off = 0
    
    def on_off(self):
        if self.on_off == 0:
            self.on_off = 1
        else:
            self.on_off = 0
            
class Radio(Device):
    
    def __init__(self):
        super().__init__()
        self.volume = 15
        
    def adjust_volume(self, delta):
        self.volume += delta
        
        

In [138]:
device = Radio()
control = ControlVolume(device)

In [139]:
device.volume

15

In [140]:
control.adjust_volume(30)

In [141]:
device.volume

45

# Decorator Pattern

- This patterns lets you attach new behaviors to objects by placing these objects inside special wrapper objects that contain the behaviors.
- **Wrapper**: object that can be linked with some target object.
- The wrapper contains the same set of methods as the target and delegates to it all requests it receives. It does some extra work before or after it passes the request to the target.

In [13]:
class Component():
    """
    The base Component interface defines operations that can be altered by
    decorators.
    """

    def operation(self, message) -> str:
        print(message)
    
class Decorator:
    
    def __init__(self, component):
        self._component = component
    
    # Uses the original component for the operation
    def operation(self, message):
        new_message = message.upper()
        self._component.operation(new_message)        

In [8]:
printer = Component()
printer.operation("Test Message")

Test Message


In [11]:
decorated_printer = Decorator(Component())
decorated_printer.operation("Text Message")

TEXT MESSAGE


Limitation: if we use a second decorator, it will not have access to the message before it is filtered by the first decorator. While we can stack decorators, we do not have access to all stages of the input.

# Abstract Factory Pattern

- In traditional OOP, "factory" is the name of a class that offers a method that builds an object. This is mostly useful when we are not allowed to pass callables as arguments, so we pass an object with an useful method instead. Since functions are first-class objects in Python, this pattern is not used a lot.

In [5]:
from abc import ABCMeta, abstractmethod

class AbstractFactory(metaclass=ABCMeta):

    @abstractmethod
    def build_sequence(self):
        pass

    @abstractmethod
    def build_number(self, string):
        pass

In [6]:
class Factory(AbstractFactory):
    def build_sequence(self):
        return []

    def build_number(self, string):
        return Decimal(string)

In [7]:
class Loader(object):
    def load(string, factory):
        sequence = factory.build_sequence()
        for substring in string.split(','):
            item = factory.build_number(substring)
            sequence.append(item)
        return sequence

f = Factory()
result = Loader.load('1.23, 4.56', f)
print(result)

[Decimal('1.23'), Decimal('4.56')]


# Builder Pattern

In [37]:
from abc import ABC, abstractmethod, abstractproperty

# Just a template of methods a Builder must have
class Builder(ABC):
    
    @abstractproperty
    def product(self):
        pass
    
    @abstractmethod
    def produce_part_a(self):
        pass
    
    @abstractmethod
    def produce_part_b(self):
        pass

In [48]:
# Provides concrete implementation of building steps. 
# This Builder is associated with Product1
class ConcreteBuilder1(Builder):
    
    # A builder instance has an empty product by default
    def __init__(self):
        self.reset()
    
    # Get an empty product. We add features to this product.
    def reset(self):
        self._product = Product1()
        
    # Builder core methods: these add parts to the associated
    # product. We add characteristics to the Product1 instance
    # in this case.
    def produce_part_a(self):
        self._product.add("PartA1")
        
    def produce_part_b(self):
        self._product.add("PartB1")
        
    # It is common practice that returns the product built so far
    # and resets the builder with a blank new product. This is an
    # optional part of the architecture
    @property
    def product(self):
        product = self._product
        self.reset()
        

In [30]:
# Empty product. We add features to an instantiated object of this
# class using a builder
class Product1():
    
    def __init__(self):
        self.parts = []
        
    def add(self, part):
        self.parts.append(part)
        
    def list_pats(self):
        print(f"Product parts: {', '.join(self.parts)}", end="")

In [ ]:
# This class is responsible for executing the building steps
# in a particular sequence. It is optional - the clinet can
# control the building steps directly
class Director:
    
    def __init__(self):
        self._builder = None
    
    # The right way to allow the user to choose the builder
    # for a director intance
    @property
    def builder(self):
        return self._builder
    
    
    # Types of builts the director is able to produce
    def build_minimal_viable_product(self):
        self.builder

# Factory Method Pattern

**A factory is an object for creating other objects**

- The **Product** declares the interface, which is common to all objects that can be produced by the creator and its subclasses. The interface should declare methods that make sense in every product.
- The **Creator** class declares the factory method that returns new product objects. The return type of this method must match the product interface. The Creator object usually contains business logic that MAKES USE of the product.

The client code works with an interface defined by Creator. We can create different factories based on this interface. These other factories will generate different products, but the client code works with them just as before.

In [23]:
# Creator class declares a factory method that is supposed to 
# return an object of a Product class. 
class Creator:
    
    # The creator may provide some default implementation
    # of the factory method
    @abstractmethod
    def factory_method(self):
        pass
        
    
    def some_operation(self):
        # Call the factory method
        product = self.factory_method()
        
        # The creator usually contains some core business logic
        # that relies on Product objects, returned by the factory method
        
        # We need method_a and method_b from any product we decide
        # to use. We will make sure anything produced by a factory
        # implements these two methods
        product.method_a()
        product.method_b()

In [24]:
class ConcreteCreator1(Creator):
    
    # In this case, it returns one of the concrete products
    # The operation is now ready to be used
    def factory_method(self):
        return ConcreteProduct1()

In [15]:
# The product interface declares the operations that all concrete
# products must implement. In this example, all products must implement
# method_a and method_b
class Product:
    
    @abstractmethod
    def method_a(self):
        pass
    
    @abstractmethod
    def method_b(self):
        pass
    

In [17]:
class ConcreteProduct1(Product):
    
    def method_a(self):
        print("Method A is working correctly")
        
    def method_b(self):
        print("Method B is working correctly")

In [20]:
creator = ConcreteCreator1()

In [21]:
creator.some_operation()

Method A is working correctly
Method B is working correctly


# Prototype Pattern

In [13]:
class Prototype:

    value = "default"

    def clone(self, **attrs):
        """Clone a prototype and update inner attributes dictionary"""
        # Python in Practice, Mark Summerfield
        obj = self.__class__()
        obj.__dict__.update(attrs)
        return obj

Use copy method (copy.copy and copy.deepcopy) for shallow and deep copies. Deep copies are useful for objects that are self-referencing in a way. You can define your own \_\_copy\_\_ and \_\_deepcopy\_\_ that will be called by the functions of the copy module.

In [14]:
class SelfReferencingEntity:
    def __init__(self):
        self.parent = None

    def set_parent(self, parent):
        self.parent = parent


class SomeComponent:
    """
    Python provides its own interface of Prototype via `copy.copy` and
    `copy.deepcopy` functions. And any class that wants to implement custom
    implementations have to override `__copy__` and `__deepcopy__` member
    functions.
    """

    def __init__(self, some_int, some_list_of_objects, some_circular_ref):
        self.some_int = some_int
        self.some_list_of_objects = some_list_of_objects
        self.some_circular_ref = some_circular_ref

    def __copy__(self):
        """
        Create a shallow copy. This method will be called whenever someone calls
        `copy.copy` with this object and the returned value is returned as the
        new shallow copy.
        """

        # First, let's create copies of the nested objects.
        some_list_of_objects = copy.copy(self.some_list_of_objects)
        some_circular_ref = copy.copy(self.some_circular_ref)

        # Then, let's clone the object itself, using the prepared clones of the
        # nested objects.
        new = self.__class__(
            self.some_int, some_list_of_objects, some_circular_ref
        )
        new.__dict__.update(self.__dict__)

        return new

    def __deepcopy__(self, memo={}):
        """
        Create a deep copy. This method will be called whenever someone calls
        `copy.deepcopy` with this object and the returned value is returned as
        the new deep copy.

        What is the use of the argument `memo`? Memo is the dictionary that is
        used by the `deepcopy` library to prevent infinite recursive copies in
        instances of circular references. Pass it to all the `deepcopy` calls
        you make in the `__deepcopy__` implementation to prevent infinite
        recursions.
        """

        # First, let's create copies of the nested objects.
        some_list_of_objects = copy.deepcopy(self.some_list_of_objects, memo)
        some_circular_ref = copy.deepcopy(self.some_circular_ref, memo)

        # Then, let's clone the object itself, using the prepared clones of the
        # nested objects.
        new = self.__class__(
            self.some_int, some_list_of_objects, some_circular_ref
        )
        new.__dict__ = copy.deepcopy(self.__dict__, memo)

        return new

# Singleton Pattern

- A class has only one instance + global access point to this instance
- Use the Singleton pattern when a class in your program should have just a single instance available to all clients
- Example: a single database access object shared by different parts of the program
- In Python, modules can also be considered a kind of singleton, albeit weaker. Whenever we import a module, it is executed. If we import it again, Python returns the imported module.

In [ ]:
# What the Gang of Four’s original Singleton Pattern
# might look like in Python.

class Logger(object):
    _instance = None

    def __init__(self):
        raise RuntimeError('Call instance() instead')

    @classmethod
    def instance(cls):
        if cls._instance is None:
            print('Creating new instance')
            cls._instance = cls.__new__(cls)
            # Put any initialization here.
        return cls._instance

## \_\_new\_\_

If we don't define a \_\_new\_\_ method, the class will have access to the parent method:

In [87]:
class A(object):
    pass

In [90]:
A.__new__

<function object.__new__(*args, **kwargs)>

We can specify our own \_\_new\_\_ method:

In [97]:
class A(object):
    # this is a class method without the decorator!
    def __new__(cls):
        print(cls)

\_\_new\_\_ is called whenever we instantiate the class. Notice that the passed argument is a reference to the class, and not an instance (it is like a class method without the need of a decorator):

In [96]:
A()

<class '__main__.A'>


In [103]:
class A(object):    
    def __init__(self, arg):
        self.arg = arg        

Creating an object is equivalent to the following operations:

In [104]:
tmp = A.__new__(A, 'an arg')
tmp.__init__('an arg')
a = tmp

print(a)
print(a.arg)

an arg


## Singleton Implementation

In [114]:
class Logger(object):
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            # this line is equivalent to
            # cls._instance = object.__new__(cls)
            # the importance here is to use something different
            # thant cls.__new__ (this would cause infinite recursion)
            cls._instance = super(Logger, cls).__new__(cls)
            # Put any initialization here.
        return cls._instance

In [115]:
Logger()

# Composite Pattern
- Tree-like pattern. The components have a common interface, so the client does not know whether it's working with a leaf or a composite object.
- Symmetry among the objects. Containers and leafs must have the same methods (related to the point above)

# Decorator Pattern

- A wrapper is linked to some target object. The wrapper constains the same set of methods as the target and delegates to it all requests it receives. The wrapper may alter the result by doing something before or after it passes the request to the target.
- From the client perspective, wrapper and target are identical.
- Some examples of applications:
    - Log method calls that would normally work silently
    - Perform extra setup or cleanup aroung a method
    - Pre-process method arguments
    - Post-process return values
    - Forbid actions that the wrapped object would normally allow

# Flyweight Pattern

# Iterator Pattern